# Amazon Recommendation System
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNd5lx4VKP1BTj0F15L_MH7UMd8dpETFudDw&usqp=CAU)
The dataset was gotten from [kaggle](https://www.kaggle.com/saurav9786/recommender-system-using-amazon-reviews/data)

Amazon seems to use both similarity score and product reviews to generate recommendataions

### Declare important modules

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("dataset/ratings_Electronics.csv")
data.head(5)

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


### Data Clean/Wrangling

In [3]:
data.columns = ['user_id', 'product_id', 'ratings', 'timestamp']
data.head(5)

,user_id,product_id,ratings,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [4]:
data.dtypes

user_id        object
product_id     object
ratings       float64
timestamp       int64
dtype: object

In [5]:
# select sample because dataset is very large
data_sample = data[:int(len(data) * .1)]

In [6]:
# prepping data for recommendation system
counts = data_sample['user_id'].value_counts()# counts how many times each unique user id appears
data = data_sample[data_sample['user_id'].isin(counts[counts >= 50].index)]

data.groupby('product_id')['ratings'].mean().sort_values(ascending=False)
final_ratings = data.pivot(index = 'user_id', columns = 'product_id', values = 'ratings').fillna(0)

num_of_ratings = np.count_nonzero(final_ratings)
possible_ratings = final_ratings.shape[0] * final_ratings.shape[1]
density = (num_of_ratings/possible_ratings)
density *= 100
final_ratings_T = final_ratings.transpose()

grouped = data.groupby('product_id').agg({'user_id': 'count'}).reset_index()
grouped.rename(columns = {'user_id': 'score'},inplace=True)
training_data = grouped.sort_values(['score', 'product_id'], ascending = [0,1]) 
training_data['Rank'] = training_data['score'].rank(ascending=0, method='first') 
recommendations = training_data.head()

### Amazon Recommendation Algorithm

In [7]:
# Designed the recommendation Algorithm based on score of the product reviews
# The algorithm takes in the user id and returns recommended products
def amazon_recommend_sys(userId):
    recommend_products = recommendations
    recommend_products['user_id'] = userId
    column = recommend_products.columns.tolist()
    column = column[-1:] + column[:-1]
    recommend_products = recommend_products[column]
    return recommend_products

print(amazon_recommend_sys(0))

      user_id  product_id  score  Rank
113         0  B00004SB92      6   1.0
1099        0  B00008OE6I      5   2.0
368         0  B00005AW1H      4   3.0
612         0  B0000645C9      4   4.0
976         0  B00007KDVI      4   5.0


/var/folders/hw/l32rftx13wsgkl0jqqgnddx00000gn/T/ipykernel_87193/370007008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_products['user_id'] = userId
